# Base ML method

## Get data

In [1]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder,OrdinalEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import time
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report,roc_curve, auc,precision_recall_curve,f1_score,balanced_accuracy_score

In [2]:
import zipfile
path_to_zip_file='/content/drive/My Drive/Data/home-credit-default-risk.zip'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Data/home-credit-default-risk.zip'

In [ ]:
# Training data
app_train = pd.read_csv('/content/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

Training data shape:  (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
app_train.dtypes

SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
                               ...   
AMT_REQ_CREDIT_BUREAU_DAY     float64
AMT_REQ_CREDIT_BUREAU_WEEK    float64
AMT_REQ_CREDIT_BUREAU_MON     float64
AMT_REQ_CREDIT_BUREAU_QRT     float64
AMT_REQ_CREDIT_BUREAU_YEAR    float64
Length: 122, dtype: object

## Feature engineer

In [ ]:
print(app_train['TARGET'].value_counts())
print(app_train['TARGET'].value_counts(normalize=True))

0    282686
1     24825
Name: TARGET, dtype: int64
0    0.919271
1    0.080729
Name: TARGET, dtype: float64


In [ ]:
dataX = app_train.drop(['TARGET'],axis=1)
dataY = app_train['TARGET']

X_train, X_test, y_train, y_test = \
    train_test_split(dataX, dataY, test_size=0.20, \
                    random_state=2018, stratify=dataY)

In [ ]:
X_train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
81406,194391,Cash loans,F,N,Y,0,157500.0,331834.5,14746.5,252000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.019689,-19699,-1703,-10039.0,-3032,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,8,0,0,0,0,0,0,Self-employed,...,NaN,0.5275,NaN,0.0124,NaN,block of flats,0.4305,Panel,No,7.0,1.0,7.0,1.0,-2300.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
168321,295110,Cash loans,F,N,Y,0,112500.0,760225.5,30280.5,679500.0,Family,Working,Secondary / secondary special,Married,House / apartment,0.006852,-21356,-4509,-4571.0,-4574,NaN,1,1,1,1,1,0,Sales staff,2.0,3,3,TUESDAY,10,0,0,0,0,0,0,Trade: type 7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.0,8.0,0.0,-2157.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
258740,399428,Cash loans,M,Y,Y,2,135000.0,152820.0,16344.0,135000.0,Family,Working,Secondary / secondary special,Married,House / apartment,0.020246,-13454,-3046,-3036.0,-5023,18.0,1,1,1,1,0,0,Drivers,4.0,3,3,TUESDAY,9,0,0,0,0,1,0,Industry: type 1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,-2050.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
271521,414775,Cash loans,F,N,Y,0,112500.0,755190.0,38686.5,675000.0,Unaccompanied,State servant,Secondary / secondary special,Married,House / apartment,0.007020,-18205,-11653,-8765.0,-1300,NaN,1,1,1,1,1,0,Core staff,2.0,2,2,WEDNESDAY,14,0,0,0,0,0,0,Kindergarten,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-384.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
82811,196046,Cash loans,F,N,Y,0,112500.0,127350.0,7920.0,112500.0,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,0.025164,-21263,365243,-871.0,-4519,NaN,1,0,0,1,1,0,NaN,1.0,2,2,TUESDAY,14,0,0,0,0,0,0,XNA,...,0.077,0.0892,0.0,0.0000,reg oper account,block of flats,0.0755,Panel,No,2.0,0.0,2.0,0.0,-659.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0


## Label encoder (Error)

Label encoder object

In [ ]:
categorical_features = X_train.select_dtypes(include=['object']).columns
for feature in categorical_features:
    #train[feature]=pd.Categorical(train[feature])
    X_train[feature]=str(X_train[feature].values)
    le=LabelEncoder()
    le.fit(X_train[feature])
    X_train[feature]=le.transform(X_train[feature])
    X_test[feature] =le.transform(X_test[feature])

Problem: different string/category in train and test set so cannot apply label encoder from train to test

Label encoder numeric (for all numeric column <=7 unique value)

In [ ]:
numeric_features_special = X_train.select_dtypes(exclude=['object']).columns
for feature in numeric_features_special:
    if len(list(X_train[feature].unique())) <= 7:
        #train[feature]=pd.Categorical(train[feature])
        X_train[feature]=str(X_train[feature].values)
        X_train[feature]=LabelEncoder().fit_transform(X_train[feature])
        X_test[feature]=LabelEncoder().transform(X_test[feature])

In [ ]:
X_train

In [ ]:
start=time.time()
lgbm_2=LGBMClassifier()

lgbm_2.fit(X_train, y_train)
end=time.time() 
print("time: ",end-start) # 9s vs 33 s H2o light gbm

In [ ]:
lgbm_2

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_pred=lgbm_2.predict(X_test)
y_pred_probs=lgbm_2.predict_proba(X_test)
print(sk.metrics.confusion_matrix(y_pred,y_test))
print("f1 score",f1_score(y_test,y_pred))
print("balanced_accuracy_score:",balanced_accuracy_score(y_test,y_pred))
fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_probs[:,1])
print("AUC:",auc(fpr, tpr))
print('classification_report \n',classification_report(y_test, y_pred))

[[56476  4882]
 [   62    83]]
f1 score 0.0324853228962818
balanced_accuracy_score: 0.5078102057712915
AUC: 0.7541847657861281
classification_report 
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     56538
           1       0.57      0.02      0.03      4965

    accuracy                           0.92     61503
   macro avg       0.75      0.51      0.50     61503
weighted avg       0.89      0.92      0.88     61503



### Pipeline

In [ ]:
categorical_features = X_train.select_dtypes(include=['object']).columns
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
impute = sk.impute.SimpleImputer()
label_transformer = LabelEncoder()
ordinal_transformer= OrdinalEncoder()
onehot_transformer= OneHotEncoder()

In [ ]:
transformers=[
    ('impute', impute, X_train.columns),         
    ('ordinal_1', ordinal_transformer, categorical_features),
    ('ordinal_2', ordinal_transformer, numeric_features),
    ('onehot', onehot_transformer, categorical_features)
    ]
preprocessor = ColumnTransformer(transformers)

In [ ]:
# Create transform X value
X_train_tran = preprocessor.fit_transform(X_train)
X_test_tran=preprocessor.transform(X_test)

## Category convert (like factor R)

In [ ]:
categorical_features = X_train.select_dtypes(include=['object']).columns
for feature in categorical_features:
    #train[feature]=pd.Categorical(train[feature])
    X_train[feature]=X_train[feature].astype("category")
    X_test[feature]=X_test[feature].astype("category")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
numeric_features_special = X_train.select_dtypes(exclude=['object']).columns
for feature in numeric_features_special:
    if len(list(X_train[feature].unique())) <= 7:
        #train[feature]=pd.Categorical(train[feature])
        X_train[feature]=X_train[feature].astype("category")
        X_test[feature]=X_test[feature].astype("category")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
start=time.time()
lgbm_2=LGBMClassifier()

lgbm_2.fit(X_train, y_train)
end=time.time() 
print("time: ",end-start) # 9s vs 33 s H2o light gbm

time:  16.821610689163208


In [ ]:
y_pred=lgbm_2.predict(X_test)
y_pred_probs=lgbm_2.predict_proba(X_test)
print(sk.metrics.confusion_matrix(y_pred,y_test))
print("f1 score",f1_score(y_test,y_pred))
print("balanced_accuracy_score:",balanced_accuracy_score(y_test,y_pred))
fpr, tpr, thresholds = roc_curve(y_test.values, y_pred_probs[:,1])
print("AUC:",auc(fpr, tpr))
print('classification_report \n',classification_report(y_test, y_pred))

[[56476  4882]
 [   62    83]]
f1 score 0.0324853228962818
balanced_accuracy_score: 0.5078102057712915
AUC: 0.7541847657861281
classification_report 
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     56538
           1       0.57      0.02      0.03      4965

    accuracy                           0.92     61503
   macro avg       0.75      0.51      0.50     61503
weighted avg       0.89      0.92      0.88     61503



In [ ]:
lgbm_2.plot_importance

AttributeError: ignored